In [ ]:
import numpy as np
import torch

# `torch.Tensor` basic

In [ ]:
# default torch.FloatTensor 
a = torch.Tensor(5, 5) 

# one way to generate tensor with specified dtype 
b = torch.DoubleTensor(5, 5) 

# generate tensor with values of zero 
c = torch.zeros([2, 4], dtype=torch.int32)
 
# specified device
d = torch.ones([2, 4], dtype=torch.int32, device = "cuda")

# numpy -> torch.tensor 
e = torch.tensor(np.array([[1, 2, 3], [4, 5, 6]])) 

# torch.tensor -> numpy 
c = c.numpy() 

# cpu -> gpu 
e = e.to("cuda")  # AVOID USING torch.cuda() 

# gpu -> cpu 
e = e.to("cpu") 


# Manipulate `torch.Tensor` shape

In [ ]:
t = torch.zeros(100, 200 ,3)
print(t.shape)
t = t.unsqueeze(0)
print(t.shape)
t = t.squeeze(0)
print(t.shape)
t = t.permute((1,0,2))
print(t.shape)
t = t.flatten()
print(t.shape)
t = t.reshape(300, -1)
print(t.shape)

# `torch.Tensor` operations (remind)

In [ ]:
a = torch.Tensor(10, 5)
b = torch.Tensor(5, 20)
c = torch.Tensor(10, 5)

# element-wisely multiply
d = a * c
print(d.shape)

# dot
e = a@b
print(e.shape)

# repeat
x = torch.Tensor(3, 4)
print(x.repeat(4, 2).shape)

# `torch.Tensor` requires_grad

In [ ]:
x = torch.tensor([[1.0, 0.0], [-1.0, 1.0]], requires_grad = True)
print(x.grad)
z = x.pow(2).sum()
z.backward()
print(x.grad)

# Simple training example

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score

def get_example(mean = 0, std = 0.2, dim = (32, 32)):
    data = np.random.normal(mean, std, size = dim)
    label = int(std * 10)
    scale = mean
    return (data, label, scale)

def get_dataset(num = 1000):
    all_data = []
    all_label = []
    all_scale = []
    for _ in range(num):
        mean = np.random.uniform(low = -1.0, high = 1.0)
        std = np.random.uniform(low = 0.1, high = 1.0)
        data, label, scale = get_example(mean = mean, std = std)
        all_data.append(data)
        all_label.append(label)
        all_scale.append(scale)
    all_data = np.array(all_data)
    all_label = np.array(all_label)
    all_scale = np.array(all_scale)
    return {"data":all_data, "label":all_label, "scale":all_scale}

In [ ]:
class RandomDataset(Dataset): 

    def __init__(self, data_num): 
        self.data_num = data_num
        self.init_data()

    def init_data(self): 
        dataset = get_dataset(self.data_num)
        self.data = dataset["data"].astype(np.float32)
        self.label = dataset["label"].astype(np.int)
        self.scale = dataset["scale"].astype(np.float32)

    def __getitem__(self, index): 
        return self.data[index], self.label[index], self.scale[index]

    def __len__(self): 
        return self.data_num

class GoodModel(nn.Module): 
    def __init__(self, class_num): 
        super(GoodModel, self).__init__() 
        self.flatten = nn.Flatten() 
        self.MLP = nn.Sequential(
            nn.Linear(32*32, 512), 
            nn.ReLU(), 
            nn.Linear(512, 256), 
            nn.ReLU(),
            nn.Linear(256, 128), 
        ) 
        self.fc1 = nn.Linear(128, class_num)
        self.fc2 = nn.Linear(128, 1)
 
    def forward(self, x): 
        x = self.flatten(x) 
        embedding = self.MLP(x) 
        logits = self.fc1(embedding) 
        score = self.fc2(embedding) 
        return logits, score
    
    def get_embedding(self, x): 
        x = self.flatten(x) 
        embedding = self.MLP(x) 
        return embedding 


In [ ]:
training_dataset = RandomDataset(1000)
training_loader = DataLoader(training_dataset, batch_size = 32, shuffle = True)
testing_dataset = RandomDataset(1000)
testing_loader = DataLoader(testing_dataset, batch_size = 32, shuffle = False)
model = GoodModel(10)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
loss_fn1 = nn.CrossEntropyLoss()
loss_fn2 = nn.L1Loss()
epochs = 100

In [ ]:
for e in range(100):
    loss1_cum = 0
    loss2_cum = 0
    true = []
    pred = []
    
    model.train()
    for data, label, scale in (training_loader):
        optimizer.zero_grad()
        logits, score = model(data)
        loss1 = loss_fn1(logits, label)
        loss2 = loss_fn2(score.flatten(), scale)
        loss = loss1 + loss2
        loss.backward()
        optimizer.step()
        
        loss1_cum += loss1
        loss2_cum += loss2

        true += list(label)
        _, predictions = torch.max(logits, 1)
        pred += list(predictions)
        
    acc = accuracy_score(true, pred)
    print(f"[{e:3d} / epochs] Train Loss: CE {loss1_cum:.3f} L1 {loss2_cum:.3f} || Acc {acc:.3f}")
    
    model.eval()
    with torch.no_grad():
        loss1_cum = 0
        loss2_cum = 0
        true = []
        pred = []
        for data, label, scale in (testing_loader): 
            logits, score = model(data)
            loss1 = loss_fn1(logits, label)
            loss2 = loss_fn2(score.flatten(), scale)
            loss1_cum += loss1
            loss2_cum += loss2

            true += list(label)
            _, predictions = torch.max(logits, 1)
            pred += list(predictions)

        acc = accuracy_score(true, pred)
        print(f"[{e:3d} / epochs] Test Loss: CE {loss1_cum:.3f} L1 {loss2_cum:.3f} || Acc {acc:.3f}")